# Secondary Structure Property Encoder Demo

This demo creates a dataset of sequence segments dericed from a non-redundant set. The dataset contains the sequence segment, the DSSP Q8 and DSSP Q3 code of the center residue in a seuqnce segment, and a property encoding of the sequence segment.

## Imports

In [6]:
from pyspark import SparkConf, SparkContext, SQLContext
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.mappers import structureToPolymerChains
from mmtfPyspark.filters import containsLProteinChain
from mmtfPyspark.datasets import secondaryStructureSegmentExtractor
from mmtfPyspark.webFilters import Pisces
from mmtfPyspark.io import mmtfReader

## Configure Spark Context

In [7]:
conf = SparkConf() \
            .setMaster("local[*]") \
            .setAppName("SecondaryStructurePropertyEncoderDemo")

sc = SparkContext(conf = conf)

 ## Read MMTF Hadoop sequence file and 
 
 Create a non-redundant set(<=20% seq. identity) of L-protein chains

In [8]:
path = "../../resources/mmtf_reduced_sample/"
sequenceIdentity = 20
resolution = 2.0
fraction = 0.1
seed = 123

pdb = mmtfReader \
        .read_sequence_file(path, sc) \
        .flatMap(structureToPolymerChains()) \
        .filter(Pisces(sequenceIdentity, resolution)) \
        .filter(containsLProteinChain()) \
        .sample(False, fraction, seed)

## Get content

In [9]:
segmentLength = 11
data = secondaryStructureSegmentExtractor.getDataset(pdb, segmentLength).cache()
print(f"original data   : {data.count()}")

original data   : 1978


## Drop Q3 and sequence duplicates

In [10]:
data = data.dropDuplicates(["labelQ3", "sequence"]).cache()
print(f"- duplicate Q3/seq  : {data.count()}")

- duplicate Q3/seq  : 1978


## Drop sequence duplicates

In [11]:
data = data.dropDuplicates(["sequence"])
print(f"- duplicate seq  : {data.count()}")

- duplicate seq  : 1978


## Property Encoding

In [13]:
encoder = proteinSequenceEncoder(data)
data = encoder.propertyEncode()

data.printSchema()
data.show(5, False)

root
 |-- structureChainId: string (nullable = true)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = true)
 |-- labelQ3: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- features: vector (nullable = true)

+----------------+-----------+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Terminate Spark Context

In [14]:
sc.stop()